# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG / Version 2: Updates laden statt Vollimport

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

#### Gründe für die Umstellung auf Updates: Durch Einspielen der Daten in Vufind ergibt sich eine zeitliche Diskrepanz, in der Titel bereits in Aleph gelöscht sind, aber in Vufind noch bestellbar. Zudem werden die Aleph-IDs hochgezähl, denn bei jedem Einspielen werden neue Nummern vergeben. Durch die Updates bleiben die Titel weiterhin verfügbar und das Hochzählen der IDs hält sich im Rahmen. Das Skript beschleunigt sich durch Reduzierung der Abfragen am Server.
*Kleiner Nachteil: der manuelle Aufwand vergrößert sich. Mehrere Dateien müssen in Aleph eingespielt und verarbeitet werden, aber hält sich nach wie vor im vertretbaren Rahmen.*

Das Jupyter Notebook arbeitet mit Python 3.8.1 und wurde mit Visual Studio Code 1.62.3 erstellt 


#### Arbeitsschritte im Code:

> Vorarbeiten:   
  - Notwendige Pandas Libraries aufrufen
  - Serverprüfung auf funktionierende Verbindung zum Aleph-X-Server    


> Daten abholen und einlesen:   
  1. Buchhandelsdaten von Schweitzer 
  2. Aleph-Konkordanz Aleph-ID /Schweitzer ID  (erzeugt tagesaktuell per p-print-03 in Aleph)
     - Aufbereiten der Daten: Schweitzer ID extrahieren und Aleph-ID mit Nullen auffüllen
  3. Daten zusammenführen in einem df    


> Daten aufbereiten:   
  1. Buchhandelsdaten prüfen und vorbereiten
     1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind und schreiben in Datei "ges02_loeschen_1"
     2. Dublettencheck innerhalb der Buchhandelsdaten
     3. Trennung der Daten in "in Aleph vorhanden" und "neu"  
  2. Bestandsabfragen:
     1. Ganz neue Titel   
        - Bestandsabgleich durch Abfragen (GES und EBX) auf dem Aleph-Server   
     2. In Aleph vorhanden
        - Bestandsabgleich durch Abfrage EBX auf dem Aleph-Server  
     3. Exportvorbereitungen:
        1. Neue Titel
        2. In Aleph vorhandene Titel   
            Identifizierung von zu löschenden Titeln "ges02_loeschen_2" und der zu aktualisierenden
  3. Exportdateien aufbereiten:
     1. Aufbereiten neuer Titel
     2. Aufbereiten der vorhandenen Titel   


> Informationssammlung
   1. Log-Datei mit Rahmendaten wird fortlaufend geschrieben
   2. Ausgabe bestimmter Titelgruppen als csv-Datei  


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [ ]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    #für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import datetime                                                    # für das Berechnen des Updates
import numpy as np                                                 # für das Bearbeiten von Spalten
import pymarc                                                      # für das Einlesen der vorhandenen Titel im Marc-Format

### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [ ]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

In [ ]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783865058041"

reply = requests.get(test).text 

print(reply) 

<hr>


## Datensätze abholen und einlesen

### 1. Datensätze von Schweitzer

*einlesen in df*

In [ ]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

In [ ]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

In [ ]:
#df

<hr>

# Testbereich wegen Dubletten in Aleph

import pymarc

input_file = './input/export_vzg_daily_ges02.mrc'
df_ges02vufind = pd.read_fwf(input_file, encoding = 'UTF-8', sep='\n', header=None, keep_default_na=False) 
#df_ges02test.columns=["ID","Field","L","Content","Feld4", "Feld5"]
df_ges02vufind.info() 

df_ges02vufind

input_file = './input/export_vzg_daily_ges02.mrc'

reader = pymarc.MARCReader(open(input_file, 'rb'), force_utf8="True") 

for record in reader:
    print (record)
    quit()
    


df

df_ges02vufind_

# Codebereich
input_file2 = './input/ges02test'
df_ges02test = pd.read_fwf(input_file2, encoding = 'UTF-8', sep='\s\s', header=None, keep_default_na=False) 
df_ges02test.columns=["ID","Field","L","Content","Feld4", "Feld5"]
df_ges02test.info() 

df_titel = df_ges02test.loc[df_ges02test['Field'] == 331]

#loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]

#df_ges02test.sort_values("Field")
df_titel["Content"].count_values()

df_dub_titel1 = df_titel.sort_values(by=["Content", "ID"]).drop_duplicates(subset=["Content"], keep='first')

df_dub_titel1

df_titel["Content"].value_counts()

df_dub_titel2 = df_titel.sort_values(by=["Content", "ID"], ascending = False).drop_duplicates(subset=["Content"], keep=False)

df_dub_titel2



#sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='last')


# Testbereich Dubletten in Aleph-Ende


<hr>

### 2. Aleph-Konkordanz Aleph-ID / Schweitzer ID

In [ ]:
now = time.strftime('%Y%m%d')  # %H:%M:%S
input_file = './input/ids'+now      #input_file wird tagesaktuell aus Aleph gezogen und auf diesem Wege mit der entsprechenden Endung eingelesen

In [ ]:

df_alephIDs = pd.read_fwf(input_file, encoding = 'UTF-8', sep='\s\s', header=None, keep_default_na=False) 
df_alephIDs.columns=["ID","Field","L","Content"]
df_alephIDs

#Spalte 0 = Aleph-IDs müsste vor Export mit Nullen aufgefüllt werden 
#Spalte 3 = object_id aus df, hier muss zum Abgleich $$a entfernt werden


*Datenfelder aufbereiten: Schweitzer-ID extrahieren und Aleph-ID ins richtige Format bringen*

In [ ]:
df_alephIDs["object_id"] = df_alephIDs["Content"].astype(str).str.slice(start=3,stop=13).apply(int)   #Spalte mit object-Ids herausschneiden und wieder zur Zahl definieren

df_alephIDs["object_id"]

In [ ]:
#Aleph-ID-Spalte mit Nullen Auffüllen
df_alephIDs["ids"] = df_alephIDs['ID'].apply(lambda x: f"{x:09d}")
df_alephIDs["ids"]

In [ ]:
#nach Dubletten vom letzten Mal vorsichtshalber, check nach doppelten object-ids
df_alephIDs["object_id"].value_counts()

# In Aleph gibt es aber Dubletten -nachforschen. Sie haben unterschiedliche object-ids!


### 3. Daten zu einem Frame zusammenführen

In [ ]:
# Zusammenführung erfolgt einmal mit concat - erlaubt die nicht mehr verwendeten Aleph-IDs zu identifizieren 
# und einmal mit join, dann erhalte ich nur die Titel, die auch im neuen Datensatz sind zur weiteren Verarbeitung!
df_oi= df.set_index("object_id")                          #object-ID zum Index für beide Datenframes
df_aleph_oi = df_alephIDs.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)                    



<hr>

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [ ]:
x = df.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
#print('vorhandene ISBNs:', df_update["isbn_ean"].shape[0])

z = df_alephIDs["ids"].count()
y = df_update_join["ids"].count()

print('-------------------------')
print('Aleph-IDs anfangs', z)
print('Aleph-IDs nach join' , y)

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))

<hr>

# Problemlösung der Aleph-Dubletten


df_alephIDs["object_id"].value_counts

df_aleph_einzel = df_alephIDs.drop_duplicates("object_id", keep=False)  #Auslesen der Eintraege mit einzelner object_id

df_aleph_doppelt = df_alephIDs.groupby("object_id").filter(lambda g: (g.nunique() >1).any()) #Doppelte object_ids rausziehen
df_aleph_single = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='first')  #von Dubletten 1. behalten 

df_aleph_vorhanden = df_aleph_einzel.append(df_aleph_single)

<hr>

## Daten aufbereiten

<hr>

### 1. Identifizierung von Titeln in Aleph-Dubletten, die nicht mehr im Datensatz sind

In [ ]:
df_update_aleph.replace({np.nan: None})

In [ ]:
df_aleph_loeschen = df_update_aleph[df_update_aleph['isbn_ean'].isna()]  


In [ ]:
df_aleph_loeschen

In [ ]:
# Schreiben der Loeschen-datei!

with open("./output/ges02_loeschen_1", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_loeschen.index:
        fa.write(df_aleph_loeschen["ids"][i]+'GES02'+'\n')

<hr>

## 2. Dublettencheck innerhalb gelieferten Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [ ]:
df = df_update_join[df_update_join['isbn_ean'].notna()]  #jetzt alle die zu verarbeiten sind rausziehen
df.reset_index(inplace=True) 

In [ ]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [ ]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [ ]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [ ]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe

In [ ]:

df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe


In [ ]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen

In [ ]:
df_dubl_nicht_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep=False)

#Kontrolle, ob es in Dubletten Titel gibt, die schon in Aleph sind
df_dubl_nicht_einspielen["ids"].count()


In [ ]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)                                         # für weitere Bearbeitung index-reset nötig

<hr>

## 3. Überblick zu den Daten und Trennen in "in Aleph vorhanden" und "neu"

In [ ]:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
print("Zum Einspielen:", m)
print("Davon in Aleph:", n)
print("Neue Titel:", m-n)

In [ ]:
df_in_aleph = df_einspielen.dropna(subset=['ids'])
df_in_aleph.reset_index(inplace=True)
df_in_aleph.shape[0]

# zur Prüfung am GES und Ebx Bestand geprüft werden

In [ ]:
df_ganz_neu = df_einspielen[df_einspielen['ids'].isnull()]

df_ganz_neu.reset_index(inplace=True)
df_ganz_neu.shape[0]

# diese müssen ebenfalls am Ges und Ebx Bestand geprüft werden, durch Trennung der Sets schnellere Bearbeitung

<hr>

## 2. Bestandsabfragen



*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

### 1. Ganz neue Titel  

= df_ganz_neu


In [ ]:
df_ganz_neu["url_ges"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.

#### URLs für Ebooks-Katalog erzeugen 

In [ ]:
df_ganz_neu["url_ebx"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

In [ ]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_ganz_neu["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = df_ganz_neu.join(df_fwf)                    #df_result = pd.concat([df_ganz_neu, df_fwf], axis=1), funktioniert nicht mehr                  

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [ ]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_ganz_neu.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [ ]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result_neu = df_result.join(df_fwf_ebx)           #df_result_neu = pd.concat([df_result, df_fwf_ebx], axis=1)

In [ ]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_neu.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

<hr>

### 2. Titel in Aleph vorhanden

= df_in_aleph

*Hier reicht Abgleich mit Ebooks, das erworbene Bücher manuell gelöscht werden.*

In [ ]:
df_in_aleph["url_ges"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link', 'http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

df_in_aleph["url_ebx"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server


In [ ]:
with open('./input/server_responses2', 'w') as fn:  
    for url in df_in_aleph["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses2', 'r') as f:
    with open('./input/server_responses_transfered2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ges = pd.read_fwf('./input/server_responses_transfered2', names=["Abfrage_ges"])
df_result2 = df_in_aleph.join(df_fwf_ges)               #df_result2 = pd.concat([df_in_aleph, df_fwf_ges], axis=1)             

In [ ]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_in_aleph.shape[0]
y = df_fwf_ges.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

In [ ]:
df_result2['Abfrage_ges']

In [ ]:
with open('./input/server_responses_ebx2', 'w') as fn:  
    for url in df_result2["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx2', 'r') as f:
    with open('./input/server_responses_transfered_ebx2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx2 = pd.read_fwf('./input/server_responses_transfered_ebx2', names=["Abfrage_ebx"])
df_result_in_aleph = df_result2.join(df_fwf_ebx2)           #df_result_in_aleph = pd.concat([df_result2, df_fwf_ebx2], axis=1)

In [ ]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_in_aleph.shape[0]
y = df_fwf_ebx2.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

<hr>

## 3. Exportvorbereitungen 


### 1. Neue Titel

In [ ]:
df_neu_nicht_einspielen =  df_result_neu.drop(df_result_neu[(df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu')].index)          
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren, diese werden mit den vorhandenen aus Update unten in Excel geschrieben


#### Jetzt Extraktion der Titel zum Einspielen:

In [ ]:
df_neu_aleph_einspielen = df_result_neu.loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]   #das sind die komplett neuen Titel

### 2. In Aleph vorhandene Titel

*Hier ist der Fall: was im Ebooks-Katalog vorhanden ist, muss gelöscht werden*

In [ ]:
df_in_aleph_nicht_einspielen = df_result_in_aleph.drop(df_result_in_aleph[(df_result_in_aleph["Abfrage_ebx"] == 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')].index) 
df_in_aleph_nicht_einspielen

In [ ]:
# Erzeugen der Datei zum Löschen der Titel

with open("./output/ges02_loeschen_2", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_nicht_einspielen.index:
        fa.write(df_in_aleph_nicht_einspielen["ids"][i]+'GES02'+'\n')

Für die Statistik und Kontrolle zusammenführen aller Titel, die nicht eingespielt werden und Ausgabe in einer CSV

In [ ]:
df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)   

In [ ]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_gesamt_nicht_einspielen["object_id"] = df_gesamt_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_gesamt_nicht_einspielen["isbn_ean"] = df_gesamt_nicht_einspielen.isbn_ean.astype(str)
df_gesamt_nicht_einspielen = df_gesamt_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_neu_nicht_einspielen.to_csv('./output/Vorhandene_Titel_'+date+'.csv')   

#### Jetzt Extraktion der Titel zum Updaten:

In [ ]:
df_in_aleph_update = df_result_in_aleph.loc[(df_result_in_aleph["Abfrage_ebx"]== 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')] # prüfen, ob es wirklich ein Datenupdate gab, sonst nicht neu einspielen??

<hr>

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [ ]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df.shape[0]
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
m = df_in_aleph_nicht_einspielen.shape[0]
n = df_ganz_neu.shape[0]
h = df_neu_aleph_einspielen.shape[0]
j = df_neu_nicht_einspielen.shape[0]
o = df_in_aleph_update.shape[0]
g = df_in_aleph.shape[0]
k = df_alephIDs.shape[0]
z = df_neu_aleph_einspielen.shape[0]       #neu ermitteln aus neuen
l = df_aleph_loeschen.shape[0]
y = df_dubl_nicht_einspielen.shape[0]


print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:             ', x,
    '\n--------------------------------------------', 
    '\nSätze ohne Dubletten               ', a,
    '\n   Dubletten:              ', c,
    '\n   Auswahl zum Einspielen  ', b,
    '\n   in Aleph löschen        ', y,
    '\nZu prüfende Datensätze:            ', a+b,
    '\n--------------------------------------------', 
    '\nPrüfung Titel in Aleph:            ', g,
    '\n   Davon als Update        ', o,
    '\n   Davon in Aleph löschen  ', m ,    
    '\nPrüfung neue Titel    :            ', n,
    '\n   Davon ganz neu          ', h,
    '\n   Davon bereits Bestand   ', j,      
    '\n--------------------------------------------', 
    '\nTitel, die in Aleph verarbeitet werden: ', o+h,
    '\n--------------------------------------------', 
    '\nDatensätze GES02 vor Einspielen:   ', k,
    '\nNicht mehr im Export               -', l,
    '\nTitel in Aleph löschen, da Bestand -', m,
    '\nZu löschen, da Dublette            -', y,
    '\nNeue Titel für Aleph               +', h,
    '\n--------------------------------------------', 
    '\nIn Aleph nach Einspielen:          ', k-l-m-y+h,)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nGelieferte Datensätze:             " + str(x))
    log.write("\n--------------------------------------------\n")
    log.write("\nSätze ohne Dubletten               " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Davon in Aleph löeschen " + str(y))
    log.write("\n   Auswahl zum Einspielen  " + str(b))
    log.write("\nZu prüfende Datensätze:            " + str(a+b))
    log.write("\n--------------------------------------------\n")
    log.write("\nPrüfung Titel in Aleph:            " + str(g))
    log.write("\n   Davon als Update        " + str(o))
    log.write("\n   Davon in Aleph löschen  " + str(m) + "    \n")   
    log.write("\nPrüfung neue Titel    :            " + str(n))
    log.write("\n   Davon ganz neu          " + str(h))
    log.write("\n   Davon bereits Bestand   " + str(j) + "    \n")     
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel, die in Aleph verarbeitet werden: " + str(o+h))
    log.write("\n--------------------------------------------\n")
    log.write("\nDatensätze GES02 vor Einspielen:   " + str(k))
    log.write("\nNicht mehr im Export               -" + str(l))
    log.write("\nZu löschender dubletter Titel      -" + str(y))
    log.write("\nTitel in Aleph löschen, da Bestand -" + str(m))
    log.write("\nNeue Titel für Aleph               +" + str(h))
    log.write("\n--------------------------------------------\n")
    log.write("\nIn Aleph nach Einspielen:          " + str(k-l-m-y+h))
    log.write("\n--------------------------------------------\n")

<hr>

## 4. Exportdateien Aufbereiten

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

## 1. Aufbereiten der neuen Titel

In [ ]:
df_aleph_einspielen = df_neu_aleph_einspielen           #zur vereinfachten Wiederverwertung des alten Codes

In [ ]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [ ]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [ ]:
today = int(time.strftime('%Y%m%d'))
df_aleph_einspielen["coming_soon"] = np.where(df_aleph_einspielen["publication_date"].astype(int) > today+10, df_aleph_einspielen["publication_date"], np.nan) #zieht die über 10 Tage raus, brauchen nan für Umwandlung in Datum

In [ ]:
df_aleph_einspielen["publication_date_soon"] = df_aleph_einspielen["coming_soon"].astype(str).str.replace('00','01')
#df_aleph_einspielen["publication_date_soon"]

In [ ]:
df_aleph_einspielen["year"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=0,stop=4)    #einfaches Zerlegen in die Datumsbestandteile und anschließendes Zusammenfügen
df_aleph_einspielen["month"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=4,stop=6)
df_aleph_einspielen["day"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=6,stop=8)

In [ ]:
df_aleph_einspielen["full_coming_soon"] = df_aleph_einspielen["year"]+'-'+df_aleph_einspielen["month"]+'-'+df_aleph_einspielen["day"]
df_aleph_einspielen["coming_soon"] = df_aleph_einspielen["full_coming_soon"].astype(str).str.replace('nan--','') 

In [ ]:
df_aleph_einspielen["coming_soon"] 

In [ ]:
df_aleph_einspielen["published"] = np.where(df_aleph_einspielen["coming_soon"] == '', df_aleph_einspielen["publication_year"], '') #Auslesen und Kombinieren der Daten
df_aleph_einspielen["year_publ"] = df_aleph_einspielen["published"].astype(str).str.slice(start=0,stop=4)                          #da wieder .0 am Ende war, Jahreszahl ausschneidens
df_aleph_einspielen["date_combined"] = df_aleph_einspielen["year_publ"]+df_aleph_einspielen["coming_soon"]
df_aleph_einspielen['date_combined']

In [ ]:
#df_aleph_einspielen.info()

In [ ]:
df_aleph_einspielen['isbn_ean'] = df_aleph_einspielen['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_aleph_einspielen['isbn_ean']

In [ ]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [ ]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [ ]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden



#### Vorbereitungen abgeschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [ ]:
with open("./output/ges02_neu", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

<hr>

### 2. Daten für Update

*es wird anhand der Spalte "last_modified" geprüft, ob die Titel seit dem letzten Einspielen ein Update erfahren haben*


 Um die Menge zum Updaten zu reduzieren, werden nur die rausgezogen, die tatsächlich ein Aktualisierungsdatum haben

In [ ]:
tday = datetime.datetime.now()
td = datetime.timedelta(days = 7)
u = tday - td
df_in_aleph_update['timespan'] = u                                                           # Einfügen einer Spalte mit Datum vor 10 Tagen als Basis für Abfrage zu Update-Notwendigkeit
print(u)

In [ ]:
df_in_aleph_update['last_modified'] = pd.to_datetime(df_in_aleph_update.last_modified)          #Umwandlung, da Spalteninhalt object ist
df_in_aleph_update['last_modified']

In [ ]:
df_in_aleph_update['true'] = np.where(df_in_aleph_update['last_modified'] > u, df_in_aleph_update['last_modified'], np.datetime64('NaT'))
df_in_aleph_update['true']

In [ ]:
df_in_aleph_update_true = df_in_aleph_update[pd.notnull(df_in_aleph_update['true'])]

In [ ]:
df_in_aleph_update_true

#### Erstellen der Update-Export-Datei

In [ ]:
#Zwischenschritt, um für alle in Aleph das Update zu starten
#df_in_aleph_update_true = df_in_aleph_update

In [ ]:
df_in_aleph_update_true["020"] = df_in_aleph_update_true["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_in_aleph_update_true["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

In [ ]:
#Aufbereitung Personendaten

person = df_in_aleph_update_true["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_in_aleph_update_true["contributor_1"]= person[0]
df_in_aleph_update_true["contributor_2"]= person[1]
df_in_aleph_update_true["contributor_3"]= person[2]

df_in_aleph_update_true["contributor_1"]= df_in_aleph_update_true["contributor_1"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_2"]= df_in_aleph_update_true["contributor_2"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_3"]= df_in_aleph_update_true["contributor_3"].replace(np.nan, '', regex=True)


### Neue Schritte zur Aufbereitung des Erscheinungsdatums, da der Code von oben hier nicht ging - warum auch immer

In [ ]:
#today = int(time.strftime('%Y%m%d'))
#soon = today+10

In [ ]:
df_in_aleph_update_true['publ_date_repl'] = df_in_aleph_update_true['publication_date'].astype(str)

In [ ]:
df_in_aleph_update_true['publ_date_repl']

In [ ]:
df_in_aleph_update_true['publ_date_repl'] = df_in_aleph_update_true['publ_date_repl'].str.replace('00','01')

In [ ]:
df_in_aleph_update_true['publ_date_repl'].value_counts()

In [ ]:
df_in_aleph_update_true['publ_date_repl_date'] = pd.to_datetime(df_in_aleph_update_true['publ_date_repl'], errors='coerce')
df_in_aleph_update_true['publ_date_repl_date']

In [ ]:
df_in_aleph_update_true["coming_soon"] = np.where(df_in_aleph_update_true["publ_date_repl_date"] > u, df_in_aleph_update_true["publ_date_repl_date"], np.datetime64('NaT'))  #df_in_aleph_update_true['today'])

In [ ]:
df_in_aleph_update_true["coming_soon"]

In [ ]:
#df_in_aleph_update_true["coming_soon"].replace('NaT','')

In [ ]:
df_in_aleph_update_true["date_combined"] = np.where(df_in_aleph_update_true["coming_soon"].astype(str) == 'NaT', df_in_aleph_update_true["publication_year"], df_in_aleph_update_true["coming_soon"].astype(str))
df_in_aleph_update_true["date_combined"]


In [ ]:
df_in_aleph_update_true['isbn_ean'] = df_in_aleph_update_true['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_in_aleph_update_true['isbn_ean']

In [ ]:
# Bearbeitung der Felder

df_in_aleph_update_true["419b"] = df_in_aleph_update_true["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_in_aleph_update_true["419c"] = df_in_aleph_update_true["date_combined"].apply(lambda x: f"$$c{x}")                  

df_in_aleph_update_true["419"] = df_in_aleph_update_true["419b"]+df_in_aleph_update_true["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

df_in_aleph_update_true["403"] = df_in_aleph_update_true["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_in_aleph_update_true["433"] = df_in_aleph_update_true["pages"].apply(lambda x: f"433   L $$b{x}")
df_in_aleph_update_true["451"] = df_in_aleph_update_true["series"].apply(lambda x: f"451   L $$b{x}") 
df_in_aleph_update_true["520"] = df_in_aleph_update_true["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_in_aleph_update_true["540"] = df_in_aleph_update_true["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_in_aleph_update_true["656"] = df_in_aleph_update_true["cover"].apply(lambda x: f"656   L $$u{x}") 
df_in_aleph_update_true["750"] = df_in_aleph_update_true["description"].apply(lambda x: f"750   L $$a{x}") 

In [ ]:
# Schreiben der Ausgabedatei, hier kleine Unterschiede zu den neuen Titeln. Vorhandene ids und bestimmte Felder können nicht verändert sein, brauchen also nicht übernommen zu werden.

with open("./output/ges02_update", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_update_true.index:
        fa.write(df_in_aleph_update_true["ids"][i]+' 100   L $$a'+df_in_aleph_update_true["contributor_1"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 104   L $$a'+df_in_aleph_update_true["contributor_2"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 108   L $$a'+df_in_aleph_update_true["contributor_3"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 331   L $$a'+df_in_aleph_update_true["title_sep"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 335   L $$a'+df_in_aleph_update_true["subtitle_all"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["403"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["419"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["433"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["451"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["520"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["540"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["656"][i]+'$$3Cover\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["750"][i]+'\n')

<hr>

## Abschließende Dinge

In [ ]:
#Abschließender Eintrag in Log-Datei
endtime = time.strftime('%H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('\n                                     beendet ')
    log.write(endtime)
    log.write("\n============================================================\n\n")

Kopien bestimmter Daten zur Einsicht bzw. für Prüfzwecke

In [ ]:
df_aleph_einspielen.to_csv('./output/Eingespielte_Titel_'+date+'.csv') 
df_in_aleph_update_true.to_csv('./output/Update_Titel_'+date+'.csv')

df_in_aleph_nicht_einspielen.to_csv('./output/Aleph_loeschen_ebx_vorh_'+date+'.csv')

<hr>

Folgende Schritte müssen mit den Dateien ausgeführt werden: 


1. Einspielen der Datei pda_ges01 als neue Titel in Aleph, hier dann auch Export der urls und Anreicherung mit der Aleph-ID mittels "mailto_link_skript.ipynb"
2. Einspielen der DAtei pda_update als "Änderungen bestehender Datensätze in Aleph"
3. Einspielen und löschen der Titel die in ges02_loeschen_1 und ges02_loeschen_2 vorhanden sind